# Configuring Jupyter On Amazon Deep Learning AMI

Using this tutorial you will be able to install and run Jupyter using https on a Amazon Deep Learning AMI. AWS now also offers a Ubuntu version of the Deep Learning AMI, most of the following steps are compatible with both versions of the Deep Learning AMI.

### Importing Images

You can import images into Jupyter by using the following Python code, this is helpful in order to make your notebooks easy to understand or to link diagrams of your neural networks. Here we pull a file from S3 into the notebook and then display it with a Ipython.display object.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/Untitled.jpg")

Now that you have launched the AWS Deep Learning AMI, and configured Jupyter you can run the next tutorials for learning how to use Jupyter, and many other MXNet tutorials. Check out the MXNet GitHub Repo ( https://github.com/dmlc/mxnet-notebooks) for tutorials that make it easy to start using Jupyter for computer vision, natural language processing, and recommendation systems. 

### Importing MXNet Notebooks
The following steps teach you how to pull notebooks from GitHub and how to sync your newly create Jupyter notebooks to S3 for backup. You can run these AWS CLI commands from your instance command prompt in order to store and sync your notebook files to S3. In Jupyter we can use the %bash operator to let Jupyter know that we want the commands that follow to be executed using the bash shell on the filesystem.

In [ ]:
%%bash
cd /home/ec2-user/notebook
git clone https://github.com/dmlc/mxnet-notebooks/

The command above pulls the CNN MXNet tutorial from the AWS MXNet github repo and places it into Jupyter as a notebook you can use to train a CNN. You can now load this notebook from the Jupyter console.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://s3-us-west-2.amazonaws.com/mxnetjupytersetup/MxNetNotebook.jpg")

In order to use the sample notebooks you need to make sure all the notebook dependencies are installed in your enviroment, the steps for doing so are below:

In [ ]:
%%bash
sudo pip install -U scikit-learn
cd /home/ec2-user/src/anaconda3/bin
sudo ./conda install graphviz -y

After running the above command, wait till for the Kernel output before continuing. You can check if the Kernel is still busy executing a command at the top right corner of the screen. MXNet comes pre-installed on the AWS Deep Learning AMI, now by going into the mxnet-notebooks folder you can run all sort of deep learning tutorials, from handwritting reccognition to a LSTM neural network. Check them out and have fun!

### Backing Up Notebooks to S3
Backing your notebooks up to S3 is a good way to ensure that you do not lose all the hard work you have put into creating them. The steps for doing so are below:

Create a S3 bucket in the AWS console, then use the following commands to backup your notebooks to S3, remmber you need to update the bucket name jupyternotebooks to your own S3 bucket in the script below in order for it to run:

In [ ]:
%%bash
aws s3 cp /home/ec2-user/notebook/ s3://jupyternotebooks --recursive

Again wait for the Kernel to finish the command and give you an output. If successful you will now have a backup of all of your notebooks in S3 that you can pull or push to from the AWS CLI on your AWS Deep Learning AMI instance.

## Calling AWS APIs From Jupyter 

Calling AWS APIs from Jupyter uses the same techniques we have already shown, both the %%bash command and then the AWS CLI commands. Below find example code that runs Python code thorugh AWS Polly for text to speech translation.

First we need to install the AWS SDK for Python (boto3) in our Anaconda installation, the steps for doing so are below:

In [ ]:
%%bash
cd /home/ec2-user/src/anaconda3/bin
sudo ./conda install boto3 -y

Wait for the Kernel to finish the command and give you an output. If successful boto3 will be installed into your Jupyter enviroment. Now we can interact with AWS API's through the Jupyter notebook. As a sample, let try and call AWS Polly to convert some text to speech from our notebook. The code to do just that is below:

In [ ]:
"""Getting Started Example for Python 2.7+/3.3+"""
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

# Create a client using the credentials and region defined in the [default]
# section of the AWS credentials file (~/.aws/credentials).
session = Session(profile_name="default")
polly = session.client("polly")

try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text="Hello and welcome to the configuring Jupyter on Amazon Deep Learning AMI tutorial!", OutputFormat="mp3",
                                        VoiceId="Joanna")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important as the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
    with closing(response["AudioStream"]) as stream:
        output = os.path.join(gettempdir(), "speech.mp3")

        try:
            # Open a file for writing the output as a binary stream
            with open(output, "wb") as file:
                file.write(stream.read())
        except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
            sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)



The code above calls AWS Polly and then converts text to speech and stores a speech.mp3 file on the /tmp drive of the filesystem. Lets make sure that happened, by checking the filesystem for the speech.mp3 file:

In [ ]:
%%bash
cd /tmp
ls -ltr

We can see that the speech.mp3 file is now on the filesystem. Lets back it up to S3 as we did with the notebooks:

In [ ]:
%%bash
aws s3 cp --acl public-read /tmp/speech.mp3 s3://pollysamplebucket

If we go into our bucket pollysamplebucket we can now see that the speech.mp3 output for Polly is now stored in S3

### Embedding Files in a Notebook
Throughout the course of this tutorial we have added images into the notebook through python code. There are many ways to add files into a notebook. Check out the python code above the images in this post and try and post your own image inside of a notebook using the format. Below we will discuss serveral other file types you can add to your notebooks.

Below we import the speech.mp3 auto file that we just converted from text to speech by AWS Polly into our Jupyter notebook and allow the user of the notebook to play them with an Ipython.display.Audio object. This is a simple use case but it can be very handy later on for deep learning tasks that require audio, for example, natural language processing engines. 

In [ ]:
import IPython
IPython.display.display(IPython.display.Audio(url="	https://s3-us-west-2.amazonaws.com/pollysamplebucket/speech.mp3"))

After going through this tutorial you should be able to stand up a fully functional Jupyter notebook on the AWS Deep Learning AMI and use MXNet to run and create your own deep learning architectures. A few sample excercises to test your knowlage are as follows:
* Deploy an AWS Deep Learning instance
* Setup Jupyter on your own AWS Deep Learning instance 
* Create your first Jupyter notebook
* Call AWS services through the Jupyter notebook
* Import an image from S3 into your notebook
* Import an audio file from S3 into your notebook

Work Hard, Have Fun, and, Make History!